In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from selenium import webdriver
import os
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
class Zarascrape:

    def __init__ (self, url): # Declaro las variables self de la clase
        self.url = url 
        self.col_names = ["url", "description"]
        self.product_list = [] 
        
    def get_info(self, url): # saca la info de cada vestido en particular
        html = requests.get(url).content
        data = BeautifulSoup(html, "lxml")
        prod_title=data.find("h1",{"class":"product-name"})
        title=prod_title.text.replace(" Detalle","")
        prod_descr = data.find("p",{"class":"description"})
        prod_various = data.find("p",{"class":"product-color"})
        prod_color=prod_various.find("span",{"class": "_colorName"})
        prod_ref=prod_various.find("span",{"data-qa-qualifier": "product-reference"})
        prod_price=data.find_all("script",{"type":"application/ld+json"})
        for i in prod_price:
            price=i.text.split(',')[-2].lstrip('"price": ').replace('"','')
        prod_size=data.find_all("div",{"class":"size-select"})
        for all_size in prod_size:
            size=all_size.find_all("label", {'class':'product-size _product-size'}) #filtramos por tallas disponibles
            available_size=[s.text for s in size]
        prod_images=data.find_all("a",{"class":"_seoImg main-image"})
        prod_image=[e.get("href") for e in prod_images][0].lstrip("//")
        try:
            features=[title,prod_descr.text,prod_color.text,prod_ref.text, price, available_size,prod_image]
        except:
            features=["","","","","","",""]
        return features
    
    def get_attrs(self,webelement, attr='href'): 
        try: 
            return webelement.get_attribute(attr)
        except: 
            return ""
    
    def get_list(self): # List of urls para vestidos
        driver = webdriver.Chrome((ChromeDriverManager().install()))
        driver.get(self.url)
        source_data = driver.page_source
        elem = driver.find_element_by_tag_name("body")

        no_of_pagedowns = 600

        res = []

        while no_of_pagedowns:
            print(no_of_pagedowns)
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            no_of_pagedowns-=1
        div = driver.find_elements_by_class_name('product-info-item')
        for i in div:
            b=i.find_elements_by_css_selector('a')
            for c in b:
                res.append(c)
        return [self.get_attrs(e, 'href') for e in res]

    def loop_it(self): # Iterates over the url root. 
        iterable = self.get_list()
        for e in iterable:
            print (len(self.product_list))
            self.product_list.append(self.get_info(e))
            time.sleep(3)
        return self.product_list
    
url_vestidos = 'https://www.zara.com/uk/en/woman-dresses-l1066.html?v1=1180427' # Scrapea sólo vestidos
url_trousers= 'https://www.zara.com/uk/en/woman-trousers-l1335.html?v1=1180429'
url_tshirts='https://www.zara.com/uk/en/woman-tshirts-l1362.html?v1=1180422'
url_shirts='https://www.zara.com/uk/en/woman-shirts-l1217.html?v1=1180428'
# Assigns the Zarascrape class for dresses to an object
#vestidos_ls = Zarascrape(url_vestidos)

# Launches the class method
#data_vestidos = vestidos_ls.loop_it()

#print(len(data_vestidos))
#data_vestidos[:5]

#trousers=Zarascrape(url_trousers)
#data_trousers=trousers.loop_it()
#tshirts=Zarascrape(url_tshirts)
#data_tshirts=tshirts.loop_it()
shirts=Zarascrape(url_shirts)
data_shirts=shirts.loop_it()
#data_tshirts[:5]


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/almu/.wdm/chromedriver/2.46/mac64/chromedriver
600
599
598
597
596
595
594
593
592
591
590
589
588
587
586
585
584
583
582
581
580
579
578
577
576
575
574
573
572
571
570
569
568
567
566
565
564
563
562
561
560
559
558
557
556
555
554
553
552
551
550
549
548
547
546
545
544
543
542
541
540
539
538
537
536
535
534
533
532
531
530
529
528
527
526
525
524
523
522
521
520
519
518
517
516
515
514
513
512
511
510
509
508
507
506
505
504
503
502
501
500
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
398
397
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
381
380

In [3]:
# Saves the df into a .csv
#zara_base = pd.DataFrame(data_vestidos)
#zara_trousers=pd.DataFrame(data_trousers)
#zara_tshirts=pd.DataFrame(data_tshirts)
zara_shirts=pd.DataFrame(data_shirts)

In [4]:
#zara_base[zara_base[3]=='0858/456']
#zara_tshirts[6][5]

In [5]:
#zara_base.shape
zara_shirts.shape

(412, 7)

In [6]:
# Saves the df into a .csv
#zara_dresses = zara_base.to_csv("zara2.csv", index=False)
#zara_trousers.to_csv("zara_trousers.csv", index=False)
#zara_tshirts.to_csv("zara_tshirts.csv", index=False)
zara_shirts.to_csv("zara_shirts.csv", index=False)